In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/RAVDESS/SPEECH'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-01-01-01-01-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-02-01-01-02-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-01-01-01-02-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-02-01-01-01-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-01-01-02-02-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-01-01-02-01-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-02-01-02-01-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-02-02-02-02-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-03-01-01-02-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-02-02-01-01-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-02-01-02-02-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-03-02-01-01-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-02-02-01-02-21.wav
/content/drive/MyDrive/RAVDESS/SPEECH/Actor_21/03-01-03-01-02-02

In [ ]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

In [ ]:
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

def gender(g):
    if int(g[0:2]) % 2 == 0:
        return 'female'
    else:
        return 'male'

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in tqdm(glob.glob("/content/drive/MyDrive/RAVDESS/SPEECH/Actor_*/*.wav")):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]] + '_' + gender(file_name.split("-")[-1])
        feature=extract_feature(file)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=2)

In [ ]:
X_train, X_val, y_train, y_val = load_data()

100%|██████████| 1440/1440 [01:57<00:00, 12.21it/s]


In [ ]:
print((X_train.shape[0], X_val.shape[0]))

(1152, 288)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
print(f'Features extracted: {X_train.shape[1]}')

Features extracted: 180


In [ ]:
from sklearn.neural_network import MLPClassifier

model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(X_train,y_train)
print(model.score(X_train, y_train))

1.0


In [ ]:
y_pred=model.predict(X_val)
print(model.score(X_val, y_val))

0.7013888888888888


In [ ]:
df=pd.DataFrame({'Actual': y_val, 'Predicted':y_pred})

In [ ]:
df

,Actual,Predicted
0,disgust_male,disgust_male
1,sad_female,sad_female
2,happy_male,happy_male
3,fearful_male,fearful_male
4,happy_female,happy_female
...,...,...
283,surprised_female,surprised_female
284,fearful_male,fearful_male
285,fearful_male,fearful_male
286,happy_male,happy_male
